In [4]:
train.data <- read.csv(file.path("..", "data", "training_data.csv"))
test.data <- read.csv(file.path("..", "data", "test_data.csv"))

In [5]:
set.seed(100)
numeric.intensity <- as.numeric(train.data$Intensity)-1

x <- train.data[, -c(2,3)]
x$Intensity <- numeric.intensity
idx.zero.var <- apply(x, 2, var) == 0


x <- x[,!idx.zero.var]
y <- train.data$VALENCE.PLEASANTNESS

x.train = x
y.train = y

# x = predicteurs avec Intensity en numeric 
# y = que la réponse

#Test data : tout sauf les 
test <- test.data[,-c(1,2)]
test$Intensity <- as.numeric(test.data$Intensity)-1
test <- test[,!idx.zero.var]

In [6]:
library(xgboost)
train.x = xgb.DMatrix(data =  as.matrix(x.train))
train.y = y.train


Attaching package: ‘xgboost’

The following object is masked from ‘package:dplyr’:

    slice



In [7]:
library(tidyverse)
library(caret)

xgb_trcontrol = trainControl(
  method = "cv",
  number = 5,  
  allowParallel = FALSE,
  verboseIter = FALSE,
  returnData = FALSE
)

In [8]:
xgbGrid <- expand.grid(nrounds = c(100,200),  # this is n_estimators in the python code above
                       max_depth = c(10, 15, 20, 25),
                       colsample_bytree = seq(0.5, 0.9, length.out = 5),
                       ## The values below are default values in the sklearn-api. 
                       eta = c(0.1,0.2,0.3),
                       gamma=0,
                       min_child_weight = 1,
                       subsample = 1
                      )

In [9]:
set.seed(100) 

xgb_model = train(
  train.x, train.y,  
  trControl = xgb_trcontrol,
  tuneGrid = xgbGrid,
  method = "xgbTree"
)

In [6]:
#CROSS-VALIDATION
best_param <- list()
best_seednumber <- 1234
best_rmse <- Inf
best_rmse_index <- 0

set.seed(123)
for (iter in 1:10) {
  param <- list(objective = "reg:squarederror",
                eval_metric = "rmse",
                max_depth = sample(3:7, 1),
                eta = runif(1, .1, .3))
  cv.nround <-  100
  cv.nfold <-  5 # 5-fold cross-validation
  seed.number  <-  sample.int(10000, 1) # set seed for the cv
  set.seed(seed.number)
  mdcv <- xgb.cv(data = dtrain, params = param,  
                 nfold = cv.nfold, nrounds = cv.nround,
                 verbose = F, early_stopping_rounds = 50, maximize = FALSE)
  min_rmse_index  <-  mdcv$best_iteration
  min_rmse <-  mdcv$evaluation_log[min_rmse_index]$test_rmse_mean

      if (min_rmse < best_rmse) {
        best_rmse <- min_rmse
        best_rmse_index <- min_rmse_index
        best_seednumber <- seed.number
        best_param <- param
      }
}
nround = best_rmse_index
set.seed(best_seednumber)

In [9]:
best_seednumber


[1] 2014

In [8]:
xg_mod <- xgboost(data = dtrain, params = best_param, nround = nround, verbose = F)

In [33]:
test = test[,c(which(colnames(test)=="Intensity"),which(colnames(test)!="Intensity"))]
colnames(test) <- NULL

In [34]:
predictions = predict(xg_mod, xgb.DMatrix(data = as.matrix(test)))

In [36]:
submission <- data.frame(Id = 1:68, VALENCE.PLEASANTNESS = predictions)
write.csv(submission, file = "../Submissions/boosting_cv.csv", row.names = FALSE)